# Visualisation of LFP & Sleep scoring

### Load LFP and packages

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
import ephyviewer
from scipy.stats import zscore
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import mkQApp, MainViewer, TraceViewer, CsvEpochSource, EpochEncoder
import json

%matplotlib widget

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
    #dpath=Path(dpath).parent
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/forgetting/Aurelie/"


fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Open Ephys folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

Load LFPs data 

In [ ]:
folder_base = Path(dpath) 
file=Path(f'{folder_base}\continuous.dat')
DataRec = np.fromfile(file, dtype="int16")

filepath = os.path.join(folder_base.parent.parent, f'structure.oebin')
with open(filepath) as f:
    metadata = json.load(f)
samplerate=metadata['continuous'][0]['sample_rate']  
numchannel=metadata['continuous'][0]['num_channels']  
All = DataRec.reshape(-1,numchannel)

print('sample rate = ', samplerate, 'Hz')
print(numchannel, 'channels recorded')
print(round(All.shape[0]/samplerate/60), 'min of recording')

Load LFPs channels

In [ ]:
Channels = '//10.69.168.1/crnldata/waking/audrey_hay/AurelieB/LFPChannels_perMice.xlsx' 
mice=file.parts[-1].split('_')[0]
print(mice)

allchannels = pd.read_excel(Channels)

PFCch1=int(allchannels[mice][0].split(',')[0])
PFCch2=int(allchannels[mice][0].split(',')[1])
CA1ch1=int(allchannels[mice][2].split(',')[0])
CA1ch2=int(allchannels[mice][2].split(',')[1])
S1ch1=int(allchannels[mice][1].split(',')[0])
S1ch2=int(allchannels[mice][1].split(',')[1])
EMGch1=int(allchannels[mice][3])

PFC  =  All[:, PFCch1]-All[:, PFCch2] 
CA1  =  All[:, CA1ch1]-All[:, CA1ch2] 
S1  =  All[:, S1ch1]-All[:, S1ch2] 
EMG  =  All[:, EMGch1]

"""
PFC  =  All[:, 5]-All[:, 6] 
CA1  =  All[:, 29]-All[:, 30] 
S1  =  All[:, 3]-All[:, 2] 
EMG  =  All[:, 24]
"""
combined = np.stack([zscore(S1),zscore(PFC), zscore(CA1), zscore(EMG)], axis = 1)
#combined = np.stack([zscore(S1),zscore(S12),zscore(PFC),zscore(PFC2), zscore(CA1),zscore(CA12), zscore(EMG)], axis = 1)

### Score sleep manually with ephyviewer


4 stages : Wake, NREM, REM, undefined

In [ ]:
possible_labels = ['Wake_1', 'NREM_2', 'REM_3', 'undefined_4'] # Wake, NREM sleep, REM sleep, undefined states
epoch_dur = 10 # define epoch duration in sec

filename = f'{dpath}\Sleep_Scoring_{len(possible_labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(filename, possible_labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=['S1','PFC', 'CA1', 'EMG'])
view1 = TraceViewer(source=source)

view1.params['xsize']= 60
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.by_channel_params['ch0', 'color'] = '#88FF88' #FF0000 red, #00FF00 green, and #0000FF blue
view1.by_channel_params['ch1', 'color'] = '#8888FF'
view1.by_channel_params['ch2', 'color'] = '#FF8888'
view1.by_channel_params['ch3', 'color'] = '#FFFFFF'

win.add_view(view1)

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')
view2.params['xsize']= 60
view2.params['new_epoch_step']= epoch_dur

view2.by_label_params['label0', 'color']= '#ffcd69' #AW
view2.by_label_params['label1', 'color']= '#69cfff' #NREM
view2.by_label_params['label2', 'color']= '#ff69cd' #REM
view2.by_label_params['label3', 'color']= '#8f8f8f' #undefined

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 50
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize']= 60
view3.by_channel_params['ch0', 'clim'] = 1
view3.by_channel_params['ch1', 'clim'] = 1
view3.by_channel_params['ch2', 'clim'] = 1
view3.by_channel_params['ch0', 'visible'] = True
view3.by_channel_params['ch1', 'visible'] = True
view3.by_channel_params['ch2', 'visible'] = True
view3.by_channel_params['ch3', 'visible'] = False

win.add_view(view3)
win.add_view(view2)

#show main window and run Qapp
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

6 stages : AW, QW, NREM, IS, REM, undefined

In [ ]:
possible_labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur = 10 # define epoch duration in sec

filename = f'{dpath}\Sleep_Scoring_{len(possible_labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(filename, possible_labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=['S1','PFC', 'CA1', 'EMG'])
view1 = TraceViewer(source=source)

view1.params['xsize']= 60
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.by_channel_params['ch0', 'color'] = '#88FF88' #FF0000 red, #00FF00 green, and #0000FF blue
view1.by_channel_params['ch1', 'color'] = '#8888FF'
view1.by_channel_params['ch2', 'color'] = '#FF8888'
view1.by_channel_params['ch3', 'color'] = '#FFFFFF'

win.add_view(view1)

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')
view2.params['xsize']= 60
view2.params['new_epoch_step']= epoch_dur

view2.by_label_params['label0', 'color']= '#ffcd69' #AW
view2.by_label_params['label1', 'color']= '#69ffe6' #QW
view2.by_label_params['label2', 'color']= '#69cfff' #NREM
view2.by_label_params['label3', 'color']= '#cd69ff' #IS
view2.by_label_params['label4', 'color']= '#ff69cd' #REM
view2.by_label_params['label5', 'color']= '#8f8f8f' #undefined

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 50
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize']= 60
view3.by_channel_params['ch0', 'clim'] = 1
view3.by_channel_params['ch1', 'clim'] = 1
view3.by_channel_params['ch2', 'clim'] = 1
view3.by_channel_params['ch0', 'visible'] = True
view3.by_channel_params['ch1', 'visible'] = True
view3.by_channel_params['ch2', 'visible'] = True
view3.by_channel_params['ch3', 'visible'] = False



win.add_view(view3)
win.add_view(view2)


#show main window and run Qapp
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'